In [ ]:
import base64
import requests as rq
import pandas as pd
import json

# Función para conseguir el token
def get_oauth_token():
    api_key = 'z27gq3rq6adcd5ks1s0btl0ujo8yg9w9'
    secret = 'pFTBKrhj3eVd'

    message = f"{api_key}:{secret}"
    auth = "Basic " + base64.b64encode(message.encode("ascii")).decode("ascii")

    headers_dic = {
        "Authorization": auth,
        "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8"
    }

    params_dic = {
        "grant_type": "client_credentials",
        "scope": "read"
    }

    r = rq.post("https://api.idealista.com/oauth/token", headers=headers_dic, data=params_dic)

    if r.status_code != 200:
        raise Exception(f"Error en la autenticación: {r.status_code}, {r.text}")

    return json.loads(r.text)['access_token']

# Función para construir la URL
def define_search_url():
    base_url = 'https://api.idealista.com/3.5/es/'  # URL base correcta
    language = 'es'
    max_items = '50'
    operation = 'sale'
    property_type = 'homes'
    center = '40.35172,-3.65467'
    distance = '7053'

    url = (base_url +
           'search?operation=' + operation +
           '&maxItems=' + max_items +
           '&center=' + center +
           '&distance=' + distance +
           '&propertyType=' + property_type +
           '&numPage=%s' +
           '&language=' + language)
    return url

# Función para buscar en la API
def search_api(url, token):
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': 'Bearer ' + token
    }

    response = rq.post(url, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Error en la búsqueda: {response.status_code}, {response.text}")
    results = json.loads(response.text)
    return results

# Convertir resultados a DataFrame
def results_to_df(results):
    df = pd.DataFrame.from_dict(results['elementList'])
    return df

# Concatenar DataFrames
def concat_df(df, df_tot):
    df_tot = pd.concat([df_tot, df], ignore_index=True)
    return df_tot

# Flujo principal
try:
    # Obtener el token y definir la URL base
    token = get_oauth_token()
    url = define_search_url()

    # Inicializar variables para paginación y DataFrame final
    pagination = 1
    df_tot = pd.DataFrame()  # DataFrame vacío para almacenar todos los resultados

    # Obtener el primer resultado para determinar el número total de páginas
    first_search_url = url % pagination
    print("URL construida:", first_search_url)
    results = search_api(first_search_url, token)

    total_pages = results['totalPages']  # Total de páginas disponibles
    print(f"Total de páginas: {total_pages}")

    # Procesar todas las páginas disponibles
    for page in range(1, total_pages + 1):
        search_url = url % page  # Crear la URL para la página actual
        print(f"Procesando página {page} de {total_pages}")
        results = search_api(search_url, token)

        # Convertir resultados a DataFrame y concatenar
        df = results_to_df(results)
        df_tot = concat_df(df, df_tot)

    # Guardar el DataFrame completo en un archivo CSV
    df_tot.to_csv("resultados_idealista.csv", index=False, encoding='utf-8')
    print("Datos guardados en resultados_idealista.csv")

except Exception as e:
    print(f"Se produjo un error: {e}")


URL construida: https://api.idealista.com/3.5/es/search?operation=sale&maxItems=50&center=40.35172,-3.65467&distance=7053&propertyType=homes&numPage=1&language=es
Se produjo un error: Error en la búsqueda: 429, 
